# Building an Image Generator

This notebook goes over how to create a data generator that reads in a section of a dataset and trains the model sequentially. 

TensorFlow wasn't loading properly on my Windows computer, so I followed this link to create a new environment to run tensorflow things: https://medium.com/@mengjiunchiou/how-to-set-keras-with-tensorflow-with-conda-virtual-environment-on-ubuntu-5d56d22e3dc7

In [1]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

C:\Users\echel\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\echel\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\echel\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\echel\AppData\Roaming\Python\Python36\site-

In [2]:
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from skimage import io, filters, measure
from scipy import ndimage
from keras.models import Sequential
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from PIL import Image # for conversion to grayscale

Using TensorFlow backend.


In [3]:
# Data have been pre-processed already. The raw images were converted
# to grayscale and were split into 25 smaller images. Labeled images
# were created from the original images and are a series of 0 (no 
# person) and 255 (a person). The labeled images were also split
# into 25 pieces so that they would match the original images. 

# Find location of image files and labeled images
data = glob('data/raw/resized/with_people/splits/*hsv*.png')
labels = glob('data/processed/dots/with_people/splits/*.png')

# Split into the training and testing data
train_X, test_X = train_test_split(data, test_size=0.25, random_state=33)
train_Y, test_Y = train_test_split(labels, test_size=0.25, random_state=33)

In [4]:
# Create function to make a data generator for the model
def gather_images(images, labels, batch_size=10): 
    """ Takes the original and labeled images, combines them into np """
    """ arrays, and passes to model"""
    while 1: 
        for offset in range(0, len(images), batch_size): 
            X = [] # empty list for training data
            Y = [] # empty list for labels 
            for img in images[offset:offset+batch_size]: # for each image in the list
                img_temp = cv2.imread(img)
                #img_temp = Image.open(img).convert('LA')
                #img_temp = cv2.cvtColor(img_temp, cv2.COLOR_BGR2HSV)
                img_flatten = np.array(img_temp) # create np array
                X.append(img_flatten) # and add to list for X
            for lab in labels[offset:offset+batch_size]: # for each label in the list
                label_temp = io.imread(lab, as_gray=True)
                labels_temp = measure.label(label_temp)
                label_flatten = labels_temp.max() # create np array
                Y.append(label_flatten) # and add to list for y
            yield (np.array(X), np.array(Y).reshape(len(Y),1)) # yield X and y for the model

## Batch Size = 30, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling

In [6]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (5,5), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/10
325/325 [==============================]325/325 [==============================] - 180s 553ms/step - loss: 15.1383 - mean_squared_error: 15.1383 - mean_absolute_error: 0.7792

Epoch 2/10
325/325 [==============================]325/325 [==============================] - 181s 558ms/step - loss: 0.3003 - mean_squared_error: 0.3003 - mean_absolute_error: 0.2466

Epoch 3/10
325/325 [==============================]325/325 [==============================] - 192s 592ms/step - loss: 0.2916 - mean_squared_error: 0.2916 - mean_absolute_error: 0.2476

Epoch 4/10
325/325 [==============================]325/325 [==============================] - 186s 571ms/step - loss: 0.2894 - mean_squared_error: 0.2894 - mean_absolute_error: 0.2412

Epoch 5/10
325/325 [==============================]325/325 [==============================] - 191s 587ms/step - loss: 0.2864 - mean_squared_error: 0.2864 - mean_absolute_error: 0.2433



## Batch Size = 30, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling, Conv # 3 = 5x5 with 3x3 pooling

In [5]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (5,5), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/10
325/325 [==============================]325/325 [==============================] - 226s 694ms/step - loss: 1.2003 - mean_squared_error: 1.2003 - mean_absolute_error: 0.3623

Epoch 2/10
325/325 [==============================]325/325 [==============================] - 204s 627ms/step - loss: 0.2871 - mean_squared_error: 0.2871 - mean_absolute_error: 0.2426

Epoch 3/10
325/325 [==============================]325/325 [==============================] - 210s 647ms/step - loss: 0.2747 - mean_squared_error: 0.2747 - mean_absolute_error: 0.2278

Epoch 4/10
325/325 [==============================]325/325 [==============================] - 197s 605ms/step - loss: 0.2520 - mean_squared_error: 0.2520 - mean_absolute_error: 0.2117

Epoch 5/10
325/325 [==============================]325/325 [==============================] - 181s 556ms/step - loss: 0.2418 - mean_squared_error: 0.2418 - mean_absolute_error: 0.2045

Ep

## Batch Size = 50, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling, Conv # 3 = 5x5 with 3x3 pooling

In [6]:
batch_size = 50

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (5,5), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
195/195 [==============================]195/195 [==============================] - 174s 891ms/step - loss: 3.0571 - mean_squared_error: 3.0571 - mean_absolute_error: 0.5316

Epoch 2/10
195/195 [==============================]195/195 [==============================] - 173s 886ms/step - loss: 0.2951 - mean_squared_error: 0.2951 - mean_absolute_error: 0.2496

Epoch 3/10
195/195 [==============================]195/195 [==============================] - 172s 881ms/step - loss: 0.2852 - mean_squared_error: 0.2852 - mean_absolute_error: 0.2394

Epoch 4/10
195/195 [==============================]195/195 [==============================] - 177s 905ms/step - loss: 0.2722 - mean_squared_error: 0.2722 - mean_absolute_error: 0.2314

Epoch 5/10
195/195 [==============================]195/195 [==============================] - 179s 919ms/step - loss: 0.2660 - mean_squared_error: 0.2660 - mean_absolute_error: 0.2262

Epoch 6/10
195/195 [==============================]195/195 [===============

## Batch Size = 50, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling

In [7]:
batch_size = 50

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (5,5), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
195/195 [==============================]195/195 [==============================] - 178s 914ms/step - loss: 17.6068 - mean_squared_error: 17.6068 - mean_absolute_error: 0.9950

Epoch 2/10
195/195 [==============================]195/195 [==============================] - 184s 942ms/step - loss: 0.3048 - mean_squared_error: 0.3048 - mean_absolute_error: 0.2523

Epoch 3/10
195/195 [==============================]195/195 [==============================] - 177s 907ms/step - loss: 0.2909 - mean_squared_error: 0.2909 - mean_absolute_error: 0.2500

Epoch 4/10
195/195 [==============================]195/195 [==============================] - 179s 917ms/step - loss: 0.2779 - mean_squared_error: 0.2779 - mean_absolute_error: 0.2404

Epoch 5/10
195/195 [==============================]195/195 [==============================] - 182s 934ms/step - loss: 0.2717 - mean_squared_error: 0.2717 - mean_absolute_error: 0.2283

Epoch 6/10
195/195 [==============================]195/195 [=============

## Batch Size = 30, Conv # 1 = 10x10 with 5x5 pooling, Conv # 2 = 5x5 with 3x3 pooling

In [8]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (10,10), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
325/325 [==============================]325/325 [==============================] - 416s 1s/step - loss: 3.5330 - mean_squared_error: 3.5330 - mean_absolute_error: 0.4980

Epoch 2/10
325/325 [==============================]325/325 [==============================] - 425s 1s/step - loss: 0.2964 - mean_squared_error: 0.2964 - mean_absolute_error: 0.2509

Epoch 3/10
325/325 [==============================]325/325 [==============================] - 398s 1s/step - loss: 0.2901 - mean_squared_error: 0.2901 - mean_absolute_error: 0.2491

Epoch 4/10
325/325 [==============================]325/325 [==============================] - 393s 1s/step - loss: 0.2842 - mean_squared_error: 0.2842 - mean_absolute_error: 0.2399

Epoch 5/10
325/325 [==============================]325/325 [==============================] - 410s 1s/step - loss: 0.2755 - mean_squared_error: 0.2755 - mean_absolute_error: 0.2345

Epoch 6/10
325/325 [==============================]325/325 [==============================

## Batch Size = 30, Conv # 1 = 10x10 with 5x5 pooling, Conv # 2 = 10x10 with 5x5 pooling

In [9]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (10,10), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(16, (10,10), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
325/325 [==============================]325/325 [==============================] - 412s 1s/step - loss: 1.7279 - mean_squared_error: 1.7279 - mean_absolute_error: 0.3966

Epoch 2/10
325/325 [==============================]325/325 [==============================] - 406s 1s/step - loss: 0.2950 - mean_squared_error: 0.2950 - mean_absolute_error: 0.2484

Epoch 3/10
325/325 [==============================]325/325 [==============================] - 416s 1s/step - loss: 0.2912 - mean_squared_error: 0.2912 - mean_absolute_error: 0.2432

Epoch 4/10
325/325 [==============================]325/325 [==============================] - 427s 1s/step - loss: 0.2857 - mean_squared_error: 0.2857 - mean_absolute_error: 0.2410

Epoch 5/10
325/325 [==============================]325/325 [==============================] - 423s 1s/step - loss: 0.2765 - mean_squared_error: 0.2765 - mean_absolute_error: 0.2344

Epoch 6/10
325/325 [==============================]325/325 [==============================

## Batch Size = 50, Conv # 1 = 10x10 with 5x5 pooling, Conv # 2 = 10x10 with 5x5 pooling

In [10]:
batch_size = 50

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (10,10), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(16, (10,10), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
195/195 [==============================]195/195 [==============================] - 405s 2s/step - loss: 2.9963 - mean_squared_error: 2.9963 - mean_absolute_error: 0.4759

Epoch 2/10
195/195 [==============================]195/195 [==============================] - 400s 2s/step - loss: 0.2990 - mean_squared_error: 0.2990 - mean_absolute_error: 0.2558

Epoch 3/10
195/195 [==============================]195/195 [==============================] - 405s 2s/step - loss: 0.2997 - mean_squared_error: 0.2997 - mean_absolute_error: 0.2579

Epoch 4/10
195/195 [==============================]195/195 [==============================] - 434s 2s/step - loss: 0.2995 - mean_squared_error: 0.2995 - mean_absolute_error: 0.2580

Epoch 5/10
195/195 [==============================]195/195 [==============================] - 433s 2s/step - loss: 0.2990 - mean_squared_error: 0.2990 - mean_absolute_error: 0.2580

Epoch 6/10
195/195 [==============================]195/195 [==============================

## Batch Size = 30, Conv # 1 = 10x10 with 5x5 pooling, Conv # 2 = 10x10 with 5x5 pooling, Conv # 3 = 5x5 with 2x2 pooling

In [11]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (10,10), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(16, (10,10), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(32, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
325/325 [==============================]325/325 [==============================] - 474s 1s/step - loss: 0.5884 - mean_squared_error: 0.5884 - mean_absolute_error: 0.2957

Epoch 2/10
325/325 [==============================]325/325 [==============================] - 426s 1s/step - loss: 0.2975 - mean_squared_error: 0.2975 - mean_absolute_error: 0.2553

Epoch 3/10
325/325 [==============================]325/325 [==============================] - 422s 1s/step - loss: 0.2962 - mean_squared_error: 0.2962 - mean_absolute_error: 0.2539

Epoch 4/10
325/325 [==============================]325/325 [==============================] - 421s 1s/step - loss: 0.2976 - mean_squared_error: 0.2976 - mean_absolute_error: 0.2548

Epoch 5/10
325/325 [==============================]325/325 [==============================] - 438s 1s/step - loss: 0.2969 - mean_squared_error: 0.2969 - mean_absolute_error: 0.2577

Epoch 6/10
325/325 [==============================]325/325 [==============================

In [143]:
predictions = model.predict_generator(gather_images(test_X, test_Y, batch_size=batch_size), len(test_X)//batch_size)
print(predictions[0:10])

[[0.06639786]
 [0.597787  ]
 [0.05902122]
 [0.04196331]
 [0.04343889]
 [0.25613928]
 [0.04196331]
 [0.12044369]
 [0.04196331]
 [0.04298855]]


In [125]:
test_labels = next(gather_images(test_X, test_Y, batch_size=len(test_X)))[1]
print(test_labels[0:10])

[[0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
